# DATA PREPARATION

 In this notebook we will focus on preparing and cleaning our datasests for analysis by handling misssing values,removing duplicates and ensuring data consistency. the steps will mostly be done in the data_cleaning script where the results will be shown in this notebook.
 

In [1]:
import pandas as pd
import sys
import os
sys.path.append(os.path.abspath('src'))


In [2]:
# we begin by loading the data and the cleaning pipeline from the src directory 
# so as to be able to use it in the notebook
from data_cleaning import load_datasets, clean_and_audit
data=load_datasets()
# we created a dictionary to store the cleaned datasets that we will use later in the project
cleaned_storage = {}


we need to review every single dataset to be able to identify the missing values, duplicate values and outliers before performig featue engineering.

In [3]:
# dataset 1 "ai_impact_jobs_2010_2025.csv"
# FIX Issue 2: pass dataset_file so the cleaning pipeline knows
# to create binary presence flags for high-missingness text columns
# (ai_keywords, ai_skills at 67.5% missing) before filling with "Unknown".
df, report = clean_and_audit(
    data['ai_impact_jobs_2010_2025.csv'],
    "AI_Impact_Historical",
    dataset_file="ai_impact_jobs_2010_2025.csv"   # ADDED
)
cleaned_storage['ai_impact_jobs_2010_2025'] = df
display(report)


,dataset,state,rows,nulls,dupes,outliers
0,AI_Impact_Historical,Raw,5000,6754,0,264
1,AI_Impact_Historical,Cleaned,5000,0,0,264


**AI Impact Historical (2010–2025) — Cleaning Summary (Raw → Cleaned)**

Raw state: the dataset contained a large number of null values concentrated in `ai_keywords` and `ai_skills` (67.5% missing each). Duplicates were also present.

**Nulls:** text columns (`ai_keywords`, `ai_skills`) filled with `"Unknown"`. Binary flag columns `has_ai_keywords` and `has_ai_skills` created before filling to preserve the signal that AI was or was not mentioned. Numeric columns filled with median.

**Outliers:** flagged only — values retained as they represent real job market extremes.

In [4]:
# dataset 2  "AI_Impact_on_Jobs_2030.csv"
df, report = clean_and_audit(data['AI_Impact_on_Jobs_2030.csv'], "AI_Impact_Future")
cleaned_storage['ai_impact_future'] = df
display(report)


,dataset,state,rows,nulls,dupes,outliers
0,AI_Impact_Future,Raw,3000,0,0,0
1,AI_Impact_Future,Cleaned,3000,0,0,0


The dataset above had zero duplicate values,zero null values and zero outliers making it a clean dataset

In [5]:
# dataset 3  "ai_job_trends_dataset.csv"
df, report = clean_and_audit(data['ai_job_trends_dataset.csv'], "AI_Job_Trends")
cleaned_storage['ai_job_trends'] = df
display(report)


,dataset,state,rows,nulls,dupes,outliers
0,AI_Job_Trends,Raw,30000,0,0,0
1,AI_Job_Trends,Cleaned,30000,0,0,0


from the above dataset we found zero duplicate values, zero null values and zero outliers making it a clean dataset.

In [6]:
# dataset 4  "career_dataset_large.xlsx"
df, report = clean_and_audit(data['career_dataset_large.xlsx'], "Career_Dataset")
cleaned_storage['career_dataset'] = df
display(report)


,dataset,state,rows,nulls,dupes,outliers
0,Career_Dataset,Raw,5000,596,1,0
1,Career_Dataset,Cleaned,4999,0,0,0


**Career Dataset — Cleaning Summary (Raw → Cleaned)**

Raw state: 596 null values and 1 duplicate row were present in the raw data.

**Nulls:** 596 missing values resolved — text columns filled with `"Unknown"`, numeric columns filled with median.

**Duplicates:** 1 duplicate row removed (first occurrence kept).

**Outliers:** none detected.

In [7]:
# dataset 5 "Coursera.csv"
# FIX Issue 1: pass dataset_file so the clip_rating outlier strategy
# is applied consistently (was previously omitted, leaving Coursera on
# the default "flag" strategy while reviews used "cap").
df, report = clean_and_audit(
    data['Coursera.csv'],
    "Coursera_Dataset",
    dataset_file="Coursera.csv"   # ADDED
)
cleaned_storage['Coursera'] = df
display(report)


,dataset,state,rows,nulls,dupes,outliers
0,Coursera_Dataset,Raw,3404,0,0,783
1,Coursera_Dataset,Cleaned,3404,0,0,783


**Coursera Dataset — Cleaning Summary (Raw → Cleaned)**

Raw state: contained out-of-range rating values in the `rate` column.

**Outliers:** rating column clipped to valid domain `[1, 5]`. IQR capping was not used — a rating of 1 is a valid response, not a statistical outlier. A `rate_was_clipped` flag column records which rows were adjusted.

In [8]:
# dataset 6 reviews.csv
# FIX Issue 1: outlier strategy changed from IQR cap to domain clip [1, 5].
# IQR capping was incorrect for a bounded ordinal rating scale.
df, report = clean_and_audit(
    data['reviews.csv'],
    "Reviews_Dataset",
    dataset_file="reviews.csv"
)
cleaned_storage['reviews'] = df
display(report)


,dataset,state,rows,nulls,dupes,outliers
0,Reviews_Dataset,Raw,107018,0,0,4720
1,Reviews_Dataset,Cleaned,107018,0,0,4720


**Reviews Dataset — Cleaning Summary (Raw → Cleaned)**

Raw state: 4,720 values in the `label` column fell outside the valid rating range `[1, 5]`.

**Outliers:** rating column clipped to `[1, 5]` using domain bounding — not IQR. Previous commentary incorrectly described this as "removed using the IQR method". IQR is inappropriate for ordinal scales with a known valid range.

In [9]:
# dataset 7 reviews_by_course.csv
# FIX Issue 1: same as reviews.csv — clip_rating replaces IQR cap.
df, report = clean_and_audit(
    data['reviews_by_course.csv'],
    "Reviews_By_Course_Dataset",
    dataset_file="reviews_by_course.csv"
)
cleaned_storage['reviews_by_course'] = df
display(report)


,dataset,state,rows,nulls,dupes,outliers
0,Reviews_By_Course_Dataset,Raw,140320,3,3016,33804
1,Reviews_By_Course_Dataset,Cleaned,137304,0,0,33709


**Reviews by Course — Cleaning Summary (Raw → Cleaned)**

Raw state: 3,016 duplicate rows, 3 null values, and 33,804 rating values outside `[1, 5]`.

**Duplicates:** 3,016 rows removed (first occurrence kept).

**Nulls:** 3 missing values filled with `"Unknown"` (text columns).

**Outliers:** `label` column clipped to valid domain `[1, 5]` — not IQR capped.

In [10]:
# dataset 8: cbc_data
df, report = clean_and_audit(data['cbc_data.csv'], "CBC_Dataset")  
cleaned_storage['cbc_data'] = df
display(report)

,dataset,state,rows,nulls,dupes,outliers
0,CBC_Dataset,Raw,1000,0,1,0
1,CBC_Dataset,Cleaned,999,0,0,0


In [11]:
 # we need to rename this dataset's column so that our scripts can work with it
if 'skills_required' in cleaned_storage['cbc_data'].columns:
    cleaned_storage['cbc_data'] = cleaned_storage['cbc_data'].rename(
        columns={'skills_required': 'skills'}
    )
    print("✔ cbc_data: renamed skills_required → skills")

✔ cbc_data: renamed skills_required → skills


In [12]:
from Education_engineering import process_cbc_education

cleaned_storage['cbc_data'] = process_cbc_education(cleaned_storage['cbc_data'])
print("✔ cbc_data: pathway mapped to education_level")
print(cleaned_storage['cbc_data']['education_level'].value_counts())

✔ cbc_data: pathway mapped to education_level
education_level
Unknown        420
Bachelors      373
Certificate    206
Name: count, dtype: int64


### map dictionary to standardise career names across datasets
we need to do this so that we can be able to create a standardised name for each career and level of education across the datasets.

In [13]:
job_titles = set()
education_levels = set() 
skills = set() 
for key, df in cleaned_storage.items(): 
    if 'job_title' in df.columns: 
        job_titles.update(df['job_title'].unique()) 
    if 'education_level' in df.columns: 
        education_levels.update(df['education_level'].unique()) 
    if 'skills' in df.columns: 
        skills.update(df['skills'].unique())

In [14]:
# lets see the number of job titles, education levels and skills we have in the datasets
print("Number of Unique Job Titles:", len(list(job_titles)))
print("Number of Unique Education Levels:", len(set(education_levels)))
print("Number of Unique Skills:", len(set(skills)))

Number of Unique Job Titles: 665
Number of Unique Education Levels: 9
Number of Unique Skills: 1404


---
## Step 2: Job Title Standardisation & Career Family Mapping

With 665 unique raw job titles, we need to reduce these to a consistent set of standard titles so that the same role is represented the same way across all 7 datasets.  



In [15]:
from Jobs_engineering import standardize_all_jobs

cleaned_storage = standardize_all_jobs(cleaned_storage, verbose=True)


[ai_impact_jobs_2010_2025]
  Using column 'job_title' for job title standardisation
  Done - 'Other' family: 0.0%

[ai_impact_future]
  Using column 'job_title' for job title standardisation
  Done - 'Other' family: 24.5%

[ai_job_trends]
  Using column 'job_title' for job title standardisation
  Done - 'Other' family: 1.7%

[career_dataset]
  Using column 'recommended_career' for job title standardisation
  Done - 'Other' family: 16.6%

[Coursera]
  No job/career column found - skipping

[reviews]
  No job/career column found - skipping

[reviews_by_course]
  No job/career column found - skipping

[cbc_data]
  Using column 'career' for job title standardisation
  Done - 'Other' family: 26.9%


In [16]:
dataset_keys = ['ai_impact_jobs_2010_2025', 'ai_impact_future', 'ai_job_trends', 'career_dataset']
# checking each dataset that has job titles
for key in dataset_keys:
    df = cleaned_storage.get(key)
    if df is None or 'career_family' not in df.columns:
        continue

    total = len(df)
    other_count = (df['career_family'] == 'Other').sum()
    other_pct = other_count / total * 100

    print(f"\n{'='*50}")
    print(f"Dataset: {key}  ({total} rows)")
    print(f"'Other' remaining: {other_count} ({other_pct:.1f}%)")
    print(f"{'='*50}")

    display(
        df['career_family']
        .value_counts()
        .to_frame('count')
        .assign(percentage=lambda x: (x['count'] / total * 100).round(1))
    )

    others = df[df['career_family'] == 'Other']['job_title_original'].unique()
    if len(others) > 0:
        print(f"\nSample of titles still in 'Other' ({len(others)} unique):")
        print(list(others[:20]))



Dataset: ai_impact_jobs_2010_2025  (5000 rows)
'Other' remaining: 0 (0.0%)


,count,percentage
career_family,,
Technology,2488,49.8
Business And Finance,1005,20.1
Policy And Research,1002,20.0
Engineering,505,10.1



Dataset: ai_impact_future  (3000 rows)
'Other' remaining: 734 (24.5%)


,count,percentage
career_family,,
Other,734,24.5
Technology,482,16.1
Business And Finance,442,14.7
Policy And Research,316,10.5
Healthcare,274,9.1
Education,166,5.5
Arts And Media,160,5.3
Transport And Aviation,153,5.1
Hospitality And Service,141,4.7



Sample of titles still in 'Other' (5 unique):
['Security Guard', 'Construction Worker', 'Mechanic', 'Customer Support', 'Retail Worker']

Dataset: ai_job_trends  (30000 rows)
'Other' remaining: 520 (1.7%)


,count,percentage
career_family,,
Business And Finance,5416,18.1
Healthcare,5321,17.7
Engineering,5012,16.7
Arts And Media,4728,15.8
Law And Public Service,1873,6.2
Education,1603,5.3
Science,1191,4.0
Technology,1170,3.9
Policy And Research,954,3.2



Sample of titles still in 'Other' (12 unique):
['Visual merchandiser', 'Radiation protection practitioner', 'Interpreter', 'Bookseller', 'Lexicographer', 'Sub', 'Gaffer', 'Best boy', 'Conservator, furniture', 'Seismic interpreter', 'Printmaker', 'Make']

Dataset: career_dataset  (4999 rows)
'Other' remaining: 830 (16.6%)


,count,percentage
career_family,,
Business And Finance,2498,50.0
Technology,845,16.9
Other,830,16.6
Policy And Research,419,8.4
Education,407,8.1



Sample of titles still in 'Other' (2 unique):
['School Counselor', 'Data Entry Operator']


## Step 3: Education Level Standardisation

In [17]:
from Education_engineering import process_education
# this code checks the levels of education in the datasets and processes them


if 'required_education' in cleaned_storage['ai_job_trends'].columns:
    cleaned_storage['ai_job_trends'] = cleaned_storage['ai_job_trends'].rename(
        columns={'required_education': 'education_level'}
    )
    print("✔ ai_job_trends: renamed required_education → education_level")

for key, df in cleaned_storage.items():
    if 'education_level' in df.columns:
        df = process_education(df)
        cleaned_storage[key] = df
        print(f"✔ {key}: education levels processed and encoded")
    else:
        print(f"  {key}: no 'education_level' column — skipped")


✔ ai_job_trends: renamed required_education → education_level
  ai_impact_jobs_2010_2025: no 'education_level' column — skipped
✔ ai_impact_future: education levels processed and encoded
✔ ai_job_trends: education levels processed and encoded
✔ career_dataset: education levels processed and encoded
  Coursera: no 'education_level' column — skipped
  reviews: no 'education_level' column — skipped
  reviews_by_course: no 'education_level' column — skipped
✔ cbc_data: education levels processed and encoded


## Step 4: SKILLS STANDARDISATION

In [18]:
import importlib
import skills_engineering

# Reload in case the module was edited
importlib.reload(skills_engineering)

from skills_engineering import process_skills, skills_summary

FIT_DATASET = "career_dataset"

if FIT_DATASET not in cleaned_storage:
    raise ValueError(f"{FIT_DATASET} not found in cleaned_storage.")

tfidf_frames = {}

# ---- Fit on the reference dataset first ----
print(f"\nFitting TF-IDF on '{FIT_DATASET}'")

fit_df = cleaned_storage[FIT_DATASET]

df_enriched, tfidf_df, tfidf = process_skills(
    fit_df,
    is_fit_dataset=True,
    save=True
)

cleaned_storage[FIT_DATASET] = df_enriched
tfidf_frames[FIT_DATASET] = tfidf_df

if "skill_count" in df_enriched.columns:
    skills_summary(df_enriched, tfidf_df)

print("Done fitting.\n")


# ---- Now apply the fitted vectoriser to the rest ----
for name, df in cleaned_storage.items():

    if name == FIT_DATASET:
        continue

    print(f"Transforming '{name}'")

    df_enriched, tfidf_df, _ = process_skills(
        df,
        tfidf=tfidf,
        is_fit_dataset=False
    )

    cleaned_storage[name] = df_enriched
    tfidf_frames[name] = tfidf_df

    if "skill_count" in df_enriched.columns:
        skills_summary(df_enriched, tfidf_df)

    print("Done.\n")

print("All datasets processed.")


Fitting TF-IDF on 'career_dataset'
✔ Skill columns detected: ['skills']
✔ TF-IDF fitted on 4999 rows
  Vocabulary size : 10
  Features created: 10
  Top 20 features : ['accounting', 'communication', 'counseling', 'data_analysis', 'financial_analysis', 'machine_learning', 'marketing', 'ms_office', 'python', 'sql']
  Saved -> models/tfidf_skills.joblib
✔ Skill columns detected: ['skills']

===== SKILL SUMMARY =====
Rows        : 4999
Avg skills  : 1.99
Max skills  : 3
Zero skills : 0 rows
Avg active TF-IDF features per row: 1.99
Done fitting.

Transforming 'ai_impact_jobs_2010_2025'
✔ Skill columns detected: ['core_skills', 'ai_skills']
✔ TF-IDF transformed 5000 rows → 10 features
✔ Skill columns detected: ['core_skills', 'ai_skills']

===== SKILL SUMMARY =====
Rows        : 5000
Avg skills  : 5.99
Max skills  : 10
Zero skills : 0 rows
Avg active TF-IDF features per row: 1.91
Done.

Transforming 'ai_impact_future'
 No skill columns found — skipping.
 No skill columns found — returning z